In [ ]:
import datetime
import numpy as np
import pandas as pd
import time
from datetime import date, timedelta
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

# Get crypto data

In [ ]:
def get_date_list(start_date,stop_date):
    date_list = []
    curr_date = start_date
    while curr_date <= stop_date:
        date_list.append(curr_date.strftime("%d-%m-%Y"))
        curr_date += timedelta(days=1)
    return date_list

In [ ]:
start_date = date(2020,10,14)
stop_date = date(2021,12,13)
date_list = get_date_list(start_date,stop_date)
coin_list = ['bitcoin', 'ethereum', 'binancecoin', 'solana', 'cardano', 'ripple',
             'polkadot', 'dogecoin', 'terra-luna', 'avalanche-2', 'enjincoin',
             'crypto-com-chain', 'matic-network', 'wrapped-bitcoin', 'litecoin',
             'tron', 'algorand', 'chainlink', 'bitcoin-cash', 'okb', 'uniswap',
             'stellar', 'cosmos', 'compound-ether', 'maker', 'ftx-token', 'vechain',
             'aave', 'elrond-erd-2', 'ethereum-classic', 'the-sandbox',
             'decentraland', 'hedera-hashgraph', 'theta-token', 'blockstack',
             'tezos', 'leo-token', 'monero', 'fantom', 'harmony', 'eos', 'iota',
             'arweave', 'bittorrent-2', 'helium', 'loopring', 'kusama',
             'quant-network', 'thorchain', 'bitcoin-cash-sv', 'neo', 'zcash',
             'celsius-degree-token', 'kucoin-shares', 'kadena',
             'basic-attention-token', 'huobi-token', 'waves', 'chiliz', 'holotoken',
             'dash', 'curve-dao-token', 'nexo', 'compound-governance-token', 'sushi',
             'nem', 'iotex', 'gatechain-token', 'nxm', 'ravencoin', 'decred', 'wax',
             'omisego', 'livepeer', 'ankr', 'zilliqa', 'icon', 'gnosis', 'siacoin',
             'bancor', 'rocket-pool', 'telcoin', 'zencash', 'iostoken', 'golem',
             '0x', 'secret', 'swissborg', 'ontology', 'uma', 'nervos-network',
             'hive', 'digibyte', 'republic-protocol', 'wazirx', 'perpetual-protocol',
             'velas', 'syscoin', 'flex-coin', 'kava']

In [ ]:
price = np.zeros((len(date_list),len(coin_list)))
mktcap = np.zeros((len(date_list),len(coin_list)))
volume = np.zeros((len(date_list),len(coin_list)))
for t in range(390,len(date_list)):
    for i in range(len(coin_list)):
        hist = cg.get_coin_history_by_id(date=date_list[t],id=coin_list[i],localization='false')
        try:
            price[t,i] = hist['market_data']['current_price']['usd']
            mktcap[t,i] = hist['market_data']['market_cap']['usd']
            volume[t,i] = hist['market_data']['total_volume']['usd']       
        except KeyError:
            price[t,i] = price[t-1,i]
            mktcap[t,i] = mktcap[t-1,i]
            volume[t,i] = volume[t-1,i]
            print(date_list[t],coin_list[i],"EXCEPTION")
        time.sleep(1.21)
    print(date_list[t],"done")

In [ ]:
price_df = pd.DataFrame(price,index=date_list,columns=coin_list)
mktcap_df = pd.DataFrame(mktcap,index=date_list,columns=coin_list)
volume_df = pd.DataFrame(volume,index=date_list,columns=coin_list)

price_df.to_csv(r"C:\Users\zhubr\OneDrive\Desktop\ECON 419\econ419-paper-data\price-extra.csv")
mktcap_df.to_csv(r"C:\Users\zhubr\OneDrive\Desktop\ECON 419\econ419-paper-data\mktcap-extra.csv")
volume_df.to_csv(r"C:\Users\zhubr\OneDrive\Desktop\ECON 419\econ419-paper-data\volume-extra.csv")

# Get T-bill data

In [ ]:
tbill1m = pd.read_csv(r"C:\Users\zhubr\Desktop\ECON 419\econ419-paper\tbill1m.csv")
tbill1m_date_list = []
for t in range(len(tbill1m)):
    curr_date = datetime.datetime.strptime(tbill1m['Date'][t],'%d/%m/%Y')
    tbill1m_date_list.append(curr_date.strftime("%d-%m-%Y"))

In [ ]:
rfrate = np.zeros(len(date_list))
for s in range(len(date_list)):
    date_found = 0
    for t in range(len(tbill1m_date_list)):
        if date_list[s] == tbill1m_date_list[t]:
            rfrate[s] = tbill1m['1M'][t]
            date_found = 1
    if date_found == 0:
        rfrate[s] = rfrate[s-1]

In [ ]:
rfrate_df = pd.DataFrame(rfrate,index=date_list,columns=['tbill_1m'])
rfrate_df.to_csv(r"C:\Users\zhubr\Desktop\ECON 419\econ419-paper\rfrate.csv")